In [134]:
"""
Import Statements: From PyTorch RL Tutorial
"""
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import nle
import gym

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [135]:
env = gym.make('NetHackChallenge-v0', savedir=None)
env.reset()

{'glyphs': array([[2359, 2359, 2359, ..., 2359, 2359, 2359],
        [2359, 2359, 2359, ..., 2359, 2359, 2359],
        [2359, 2359, 2359, ..., 2359, 2359, 2359],
        ...,
        [2359, 2359, 2359, ..., 2359, 2359, 2359],
        [2359, 2359, 2359, ..., 2359, 2359, 2359],
        [2359, 2359, 2359, ..., 2359, 2359, 2359]], dtype=int16),
 'chars': array([[32, 32, 32, ..., 32, 32, 32],
        [32, 32, 32, ..., 32, 32, 32],
        [32, 32, 32, ..., 32, 32, 32],
        ...,
        [32, 32, 32, ..., 32, 32, 32],
        [32, 32, 32, ..., 32, 32, 32],
        [32, 32, 32, ..., 32, 32, 32]], dtype=uint8),
 'colors': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'specials': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0,

In [153]:
# Transition Class (represents change from one state to a another)
StateTransition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

# Replay Memory Class (represents a memory of an experience, or state transition)
class ReplayMemory():
    def __init__(self, max_memory):
        self.memory = []
        self.max_memory = max_memory
    
    def push(self, newExperience):
        if len(self.memory) >= self.max_memory:
            self.memory.pop(0)
        self.memory.append(newExperience)
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [154]:
class DQNetwork(nn.Module):
    def __init__(self):
        super(DQNetwork, self).__init__()
        self.joe = nn.Sequential(
            nn.Conv2d(4, 16, kernel_size=3, padding='same'),
            nn.BatchNorm2d(16),
            nn.ReLU(), # 21 * 79 * 16
            nn.MaxPool2d(3, padding=(0,1)), # 7 * 27 * 16
            nn.Conv2d(16, 64, kernel_size=3, padding='same'),# 7 * 27 * 64
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3, padding=(1,0)), # 3 * 9 * 64
            nn.Flatten(),
            nn.Linear(3*9*64, 3*64),
            nn.ReLU(),
            nn.Linear(3*64, 113),
        )
        
    def forward(self, x):
        logits = self.joe(x.float())
        return logits
        
        

In [155]:
x = np.load('l.txt.npy')
x = torch.Tensor(x)[None,:,:,:]
print(x.shape)

torch.Size([1, 4, 21, 79])


In [156]:
bink = DQNetwork()
bink.forward(x)

tensor([[ 2.8304e-01,  1.0052e-02, -5.7900e-01, -2.9673e-01, -5.6491e-02,
          2.1603e-01, -3.4159e-01, -1.8731e-01, -3.0812e-01,  1.1164e-01,
         -5.2045e-01,  1.8728e-01,  8.7429e-03,  4.8387e-04,  2.5057e-01,
          2.5009e-01, -1.3320e-01,  8.6689e-01,  3.7173e-01, -3.8553e-01,
          3.6755e-01,  3.2878e-01, -3.0470e-01, -1.1364e-01,  7.6163e-02,
         -1.2119e-01,  8.9848e-02, -2.7088e-01,  1.8923e-01,  2.3233e-01,
         -3.2015e-01,  3.9316e-02,  2.3188e-01, -4.3725e-01,  3.8906e-01,
          2.4832e-02,  4.1420e-01, -4.1497e-01, -1.0266e-01, -5.2499e-01,
          1.3781e-01, -2.5641e-02,  1.5768e-01, -2.5781e-01, -2.7788e-01,
          2.2917e-01,  8.4392e-02, -2.7387e-01, -1.3557e-01,  2.2888e-01,
         -3.0993e-01, -4.7942e-01,  1.7450e-02, -5.8284e-01, -1.9728e-01,
         -4.6851e-01,  1.0893e-01, -1.1585e-01, -1.9302e-01,  1.6277e-01,
          3.6498e-01, -2.0792e-01, -1.4542e-01,  1.6003e-01,  4.4351e-01,
          9.1785e-02,  2.2712e-01,  6.

In [157]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10
steps_done = 0
episode_durations = []
episode_rewards = []
n_actions = env.action_space.n

In [158]:
policy_net = DQNetwork().to(device)
target_net = DQNetwork().to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

In [159]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

In [160]:
def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

In [161]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = StateTransition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [162]:
def concat_state(state):
    state = np.stack((state['glyphs'], state['chars'], state['colors'], state['specials']))
    state = torch.tensor(np.expand_dims(state, axis=0), device=device)
    return state


num_episodes = 50
for i_episode in range(num_episodes):
    # Initialize the environment and state
    obs = env.reset()
    state = concat_state(obs)
    for t in count():
        # Select and perform an action
        action = select_action(state)
        obs, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        
        # Keep track of current state and previous states 
        last_state = state
        state = concat_state(obs)

        # Store the transition in memory
        memory.push(StateTransition(last_state, action, state, reward))

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

<Figure size 432x288 with 0 Axes>

Complete

                                                                                
    level 2.  Killed by a kobold.                           -  [15]             
                                                                     2         2
4  Agent-Pri-Elf-Fem-Cha quit in The Dungeons of Doom on                        
                                                                        level 1.
                                                8  [13]                         
                                                         3          8  Agent-Val
-Dwa-Fem-Law escaped the dungeon [max level                                     
                                                           1].                  
                                  18  [18]                                      
                                                                                
                                           14          1  Agent-Arc-Gno-Fem-Neu 
quit in The Dungeo

<Figure size 432x288 with 0 Axes>

In [164]:
max(episode_durations)

165571

In [167]:
rewards = [x.reward for x in memory.memory]
max(rewards)

tensor([0.], device='cuda:0')